In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time
from functools import partial
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver as webdriver
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
list_topic=['100','101','102','103','104','105']
list_subtopic=[['264','265','268','266','267','269'],#청와대,국회/정당,북한,행정,국방/외교,정치일반
               ['259','258','261','771','260','262','310','263'],#금융,증권,산업/재계,중기/벤처,부동산,글로벌 경제,생활경제,경제 일반
               ['249','250','251','254','252','59b','255','256','257'],#사건사고,교육,노동,언론,환경,인권/복지,식품/의료,지역,사회일반
               ['241','239','240','237','238','376','245'],#건강정보,자동차/시승기,도로/교통,여행/레저,음식/맛집,패션/뷰티,생활문화 일반
               ['231','232','233','234','322'],#아시아/호주,미국/중남미,유럽,중동/아프리카,세계 일반
               ['731','226','227','230','732','283','229','228'],#모바일,인터넷/sns,통신/뉴미디어,IT일반,보안/해킹,컴퓨터,게임/리뷰,과학일반
              ]
topic_dict={'100':"정치",'101':"경제",'102':"사회",'103':"생활/문화",'104':"국제",'105':"IT/과학"}
topic_sub={'264':"청와대",'265':'국회/정당','268':'북한','266':'행정','267':'국방/외교','269':'정치일반',
           '259':'금융','258':'증권','261':'산업/재계','771':'중기/벤처','260':'부동산','262':'글로벌 경제','310':'생활경제','263':'경제일반',
           '249':'사건사고','250':'교육','251':'노동','254':'언론','252':'환경','59b':'인권/복지','255':'식품/의료','256':'지역','257':'사회일반',
           '241':'건강정보','239':'자동차/시승기','240':'도로/교통','237':'여행/레저','238':'음식/맛집','376':'패션/뷰티','245':'생활문화 일반',
           '231':'아시아/호주','232':'미국/중남미','233':'유럽','234':'중동/아프리카','322':'세계 일반',
           '731':'모바일','226':'인터넷/sns','227':'통신/뉴미디어','230':'IT일반','732':'보안/해킹','283':'컴퓨터','229':'게임/리뷰','228':'과학일반'}

In [3]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20200101')

In [4]:
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',}

In [5]:
# 각각의 리스트 생성
topic_list=[]
date_list=[]
title_list=[]
text_list=[] 
subtopic_list=[]
page_list=[]

In [6]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [7]:
options=webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')



driver = webdriver.Chrome('./chromedriver',options=options)
driver2 = webdriver.Chrome('./chromedriver',options=options)
driver3 = webdriver.Chrome('./chromedriver',options=options)


driver.implicitly_wait(10)
driver2.implicitly_wait(10)
driver3.implicitly_wait(10)

driver.get("https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264")


In [8]:
def get_news(link):
    driver.get(link)
    print(link)
    get_content(driver,driver3)
    for x in driver.find_elements_by_css_selector("#main_content > div.paging > a"):
        if x.text=='다음':
            get_news(x.get_attribute('href'))
        else:
            driver2.get(x.get_attribute('href'))
            get_content(driver2,driver3)

In [9]:
def get_content(driver,driver3):
    con_list=driver.find_elements_by_css_selector("#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl ")
    con_list.extend(driver.find_elements_by_css_selector("#main_content > div.list_body.newsflash_body > ul.type06 > li > dl "))
    for i in con_list:
        x=i.find_elements_by_css_selector('dt')
        try:
            con_url=x[0].find_element_by_css_selector('a').get_attribute('href')
            print(con_url)
        except:
            pass
        print(con_url)
        driver3.get(con_url)
        title=driver3.find_element_by_css_selector("#articleTitle").text
        title=no_space(title)
        title_list.append(title)
      #  print("title: "+title)
        text=driver3.find_element_by_css_selector("#articleBodyContents").text
        text=no_space(text)
        text_list.append(text)
      #  print("text: "+text)
        emotion=(driver3.find_element_by_css_selector("#spiLayer > div._reactionModule.u_likeit ").text).split('\n')

In [10]:
#크롤링 시작
num=0
start_time=time.time()
for num_main in tqdm(range(len(list_topic)),desc='Topic_Progress',leave=False):
    for num_sub in tqdm(range(len(list_subtopic[num])),desc='Sub_Topic_Progress',leave=False):
        main_url="https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2="+list_subtopic[num_main][num_sub]+"&sid1="+list_topic[num_main]+"&date="
        for num_date in tqdm(range(len(dt_index.date)),desc='Day_Progress'):
            date=str(dt_index.date[num_date])
            date=date.split('-')[0]+date.split('-')[1]+date.split('-')[2]
            get_news(main_url+date)    

            print(date)
            time.sleep(0.05)
            
            
    num+=1
print(time.time()-start_time)

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=264&sid1=100&date=20200101
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=032&aid=0002983868
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=032&aid=0002983868
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387220
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387220
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=011&aid=0003674411
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=011&aid=0003674411
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387208
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387208
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333375
https://news.naver.com/main/read.nhn?mode=LS2D&mi

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386911
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386911
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386910
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386910
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386909
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004386909
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309389
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309389
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309388
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309388
https://news.naver.com/main/read.nhn?mod

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=018&aid=0004546974
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=018&aid=0004546974
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333244
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333244
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309168
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309168
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333242
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333242
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=014&aid=0004351191
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=014&aid=0004351191
https://news.naver.com/main/read.nhn?mod

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=003&aid=0009634131
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=003&aid=0009634131
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=003&aid=0009634130
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=003&aid=0009634130
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309097
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309097
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309096
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309096
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309095
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309095
https://news.naver.com/main/read.nhn?mod

https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309065
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309065
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309064
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309064
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309063
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309063
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309062
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309062
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309061
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011309061
https://news.naver.com/main/read.nhn?mod

...

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=254&sid1=102&date=20200101
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309585
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309585
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309391
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309391
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309333
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309333
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309317
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309317
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=254&oid=001&aid=0011309315
https://news.naver.com/main/read.nhn?mode=LS2D&mi

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=252&sid1=102&date=20200101
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=001&aid=0011309740
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=001&aid=0011309740
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=001&aid=0011309737
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=001&aid=0011309737
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=003&aid=0009634500
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=003&aid=0009634500
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=468&aid=0000610527
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=468&aid=0000610527
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=252&oid=001&aid=0011309482
https://news.naver.com/main/read.nhn?mode=LS2D&mi

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=59b&sid1=102&date=20200101
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=018&aid=0004547050
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=018&aid=0004547050
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011309259
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011309259
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011309251
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011309251
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011308927
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=001&aid=0011308927
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=59b&oid=016&aid=0001619586
https://news.naver.com/main/read.nhn?mode=LS2D&mi

https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=255&sid1=102&date=20200101
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=032&aid=0002983839
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=032&aid=0002983839
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=001&aid=0011309698
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=001&aid=0011309698
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=020&aid=0003261837
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=020&aid=0003261837
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=003&aid=0009634475
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=003&aid=0009634475
https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=102&sid2=255&oid=003&aid=0009634455
https://news.naver.com/main/read.nhn?mode=LS2D&mi

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#articleTitle"}
  (Session info: headless chrome=87.0.4280.88)


In [ ]:
print(time.time()-start_time)

In [ ]:
driver.quit()
driver2.quit()
driver3.quit()

In [ ]:
con_list=driver.find_elements_by_css_selector("#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl ")
con_list.extend(driver.find_elements_by_css_selector("#main_content > div.list_body.newsflash_body > ul.type06 > li > dl "))

print(con_list)
for i in con_list:
    x=i.find_elements_by_css_selector('dt')
    try:
        con_url=x[0].find_element_by_css_selector('a').get_attribute('href')
        print(con_url)
    except:
        pass

In [ ]:
get_news("https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=264&sid1=100&date=20201228")

In [ ]:
driver.get(driver.find_elements_by_css_selector("#main_content > div.paging > a")[0])

In [ ]:
x=driver.find_elements_by_css_selector("#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl > dt")


In [ ]:
k=x[0].find_element_by_css_selector('a').get_attribute('href')

In [ ]:
k

In [ ]:
#크롤링 def
def get_news(date,topic,sub_topic,main_url):
    
    driver.get(main_url)
    
    driver.find_element_by_css_selector("#main_content > div.list_body.newsflash_body > ul > li > dl "):
        


In [ ]:
meta=meta.select_one("a")
        link=meta['href']
        date=date
        url=link
        url=requests.get(url,headers=headers)
        bs_obj=BeautifulSoup(url.content, "html.parser")  
        try:
            title=bs_obj.select_one("#articleTitle").text         
            x=bs_obj.select_one("#articleBodyContents").text
            title=no_space(title)
            text=no_space(x)        
            date_list.append(date)
            topic_list.append(topic_dict[topic])
            subtopic_list.append(topic_sub[sub_topic])
            title_list.append(title)
            text_list.append(text)
        except:
            print(meta['href'])

In [ ]:
#크롤링 시작
num=0
start_time=time.time()
for num_main in tqdm(range(len(list_topic)),desc='Topic_Progress',leave=False):
    for num_sub in tqdm(range(len(list_subtopic[num])),desc='Sub_Topic_Progress',leave=False):
        main_url="https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2="+list_subtopic[num_main][num_sub]+"&sid1="+list_topic[num_main]+"&date="
        for num_date in tqdm(range(len(dt_index.date)),desc='Day_Progress'):
            date=str(dt_index.date[num_date])
            date=date.split('-')[0]+date.split('-')[1]+date.split('-')[2]
            page_list=[]
            get_news(date,list_topic[num_main],list_subtopic[num_main][num_sub],main_url+date)    
            
            
            html = main_url+date
            html=requests.get(html,headers=headers)
            bsObject = BeautifulSoup(html.content, "html.parser")     
            
            page=bsObject.select("#main_content > div.paging > a")
            paging(page,page_list)
            
            for k in page_list:
                get_news(date,list_topic[num_main],list_subtopic[num_main][num_sub],k)




            print(date)
            time.sleep(0.05)
            
            
    num+=1
print(time.time()-start_time)

In [ ]:
#판다스로 저장 (후에 csv파일로 저장 라인 추가)

df=pd.DataFrame({"date":date_list, "title":title_list, "topic":topic_list,"subtopic":subtopic_list,"text":text_list}) 
print(df) 

In [ ]:
url="https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=469&aid=0000567143"
url=requests.get(url,headers=headers)
bsObject = BeautifulSoup(url.content, "html.parser")

In [ ]:
k=bsObject.select("#spiLayer > div._reactionModule.u_likeit ")

In [ ]:
print(k)